# Tests with the eGPU

Table of contents:

- [Basic Checks](#basic-checks)
- [Basic Sequential Neural Network](#basic-sequential-neural-network)
- [MNIST Training Test](#mnist-training-test)

## Basic Checks

In [1]:
import torch

# How many GPUs?
print("CUDA available:", torch.cuda.is_available())
print("GPU count:", torch.cuda.device_count())

# Name of each GPU
for i in range(torch.cuda.device_count()):
    print(i, torch.cuda.get_device_name(i))

# Memory usage
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Memory allocated:", torch.cuda.memory_allocated(device) / 1024**2, "MB")
print("Max memory allocated:", torch.cuda.max_memory_allocated(device) / 1024**2, "MB")
print("Memory reserved:", torch.cuda.memory_reserved(device) / 1024**2, "MB")

# Clear cache
torch.cuda.empty_cache()

CUDA available: False
GPU count: 0
Memory allocated: 0.0 MB
Max memory allocated: 0.0 MB
Memory reserved: 0.0 MB


In [2]:
# Dosplay CLI tool output nvidia-smi
!nvidia-smi

# For live tracking in Terminal, use built-in loop option
# nvidia-smi -l 1

Sat Oct  4 11:24:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.163.01             Driver Version: 550.163.01     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA T500                    Off |   00000000:01:00.0 Off |                  N/A |
| N/A   49C    P8             N/A / ERR!  |       8MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Basic Sequential Neural Network

In [7]:
import time
import torch
import torch.nn as nn
import torch.optim as optim

if torch.cuda.is_available():
    # Pick device (force RTX 3060, index 0)
    device = torch.device("cuda:0")
    print("Using device:", device, torch.cuda.get_device_name(device))
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using device:", device)
else:
    device = torch.device("cpu")
    print("Using device:", device)

# Dummy dataset: 100,000 samples, 100 features, 10 classes
X = torch.randn(100000, 100, device=device)
y = torch.randint(0, 10, (100000,), device=device)

# Simple MLP
model = nn.Sequential(
    nn.Linear(100, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training loop
start_time = time.time()
for epoch in range(5):
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")
end_time = time.time()
print(f"Training completed in {end_time - start_time:.2f} seconds")

Using device: mps
Epoch 1, Loss: 2.3300
Epoch 2, Loss: 2.3217
Epoch 3, Loss: 2.3163
Epoch 4, Loss: 2.3130
Epoch 5, Loss: 2.3109
Training completed in 0.99 seconds


## MNIST Training Test

In [ ]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

if torch.cuda.is_available():
    # Pick device (force RTX 3060, index 0)
    device = torch.device("cuda:0")
    print("Using device:", device, torch.cuda.get_device_name(device))
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using device:", device)
else:
    device = torch.device("cpu")
    print("Using device:", device)

# Load MNIST
transform = transforms.Compose([transforms.ToTensor()])
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Simple CNN
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.fc1 = None   # we’ll initialize later
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.flatten(x, 1)
        if self.fc1 is None:  # lazy init
            self.fc1 = nn.Linear(x.shape[1], 128).to(x.device)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

start_time = time.time()
for epoch in range(10):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader):.4f}")
end_time = time.time()
print(f"Training completed in {end_time - start_time:.2f} seconds")

Using device: mps
Epoch 1, Loss: 1.0225
Epoch 2, Loss: 0.4009
Epoch 3, Loss: 0.3360
Epoch 4, Loss: 0.3063
Epoch 5, Loss: 0.2873
Epoch 6, Loss: 0.2738
Epoch 7, Loss: 0.2639
Epoch 8, Loss: 0.2558
Epoch 9, Loss: 0.2495
Epoch 10, Loss: 0.2437
Training completed in 61.50 seconds


In [5]:
# Delete tensors
del model, optimizer # X, y
# Empty any content in GPU
torch.cuda.empty_cache()